In [2]:
#https://www.cnblogs.com/DswCnblog/p/6126588.html
#usage of 'with'
'''
file = open("/tmp/foo.txt")
try:
    data = file.read()
finally:
    file.close()  
<=>
with open("/tmp/foo.txt") as file:
    data = file.read()
'''
 
class Sample:
    def __enter__(self):
        print ("In __enter__()")
        return ("Foo")
 
    def __exit__(self, type, value, trace):
        print ("In __exit__()")

with Sample() as sample:
    print ("sample:", sample)

In __enter__()
sample: Foo
In __exit__()


In [3]:
import tensorflow as tf

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import time

time_start=time.clock()

one=tf.constant(1)
x1=tf.Variable(0)
#temp=tf.add(x1,one)
x1_update=tf.assign(x1,tf.add(x1,one))

init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        sess.run(x1_update)
    print(x1.eval())
    
time_end=time.clock()
print('totally cost',time_end-time_start)

1000
totally cost 0.2016560444725175


# Running a Graph in a Session

In [4]:
x=tf.Variable(3,name='x')
y=tf.Variable(4,name='y')
f=x*x*y+y+2

In [5]:
sess=tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
print(sess.run(f))
sess.close()

42


In [6]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    print(f.eval())

42


In [7]:
init =tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    print(f.eval())

42


In [8]:
sess=tf.InteractiveSession()
init.run()
print(f.eval())
sess.close()

42


# Managing Graphs

In [9]:
x.graph is tf.get_default_graph()

True

In [10]:
graph =tf.Graph()
with graph.as_default():
    x2=tf.Variable(2)
print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

True
False


In [4]:
#equation solver
w=tf.constant(3)
x=w+2
y=x+5
z=x*3
with tf.Session() as sess:
    print(sess.run([y,z]))

[10, 15]


# Linear Regression

In [12]:
import numpy as np
from sklearn.datasets import fetch_california_housing
housing=fetch_california_housing()
m,n=housing.data.shape
housing_data_plus_bias=np.c_[np.ones((m,1)),housing.data]

X=tf.constant(housing_data_plus_bias,dtype=tf.float32)
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32)
X_T=tf.transpose(X)
theta=tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(X_T,X)),X_T),y)

with tf.Session() as sess:
    print(theta.eval())

[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


# Gradient Descent

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [14]:
tf.reset_default_graph()
n_epochs=301
learning_rate=0.1

X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32)
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32)
theta=tf.Variable(tf.random_uniform([n+1,1],-1,1))
y_pred=tf.matmul(X,theta)
error=y_pred-y
mse=tf.reduce_mean(tf.square(error))
#gradients=tf.gradients(mse,[theta])[0]
#training_op=tf.assign(theta,theta-learning_rate*gradients)
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(mse)

init=tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        sess.run(training_op)
        if epoch % 30==0:
            print('Epoch: ',epoch,'MSE=',mse.eval())
    best_theta=theta.eval()
print(best_theta)

Epoch:  0 MSE= 6.646847
Epoch:  30 MSE= 0.6654864
Epoch:  60 MSE= 0.58900315
Epoch:  90 MSE= 0.55545115
Epoch:  120 MSE= 0.53997815
Epoch:  150 MSE= 0.5324972
Epoch:  180 MSE= 0.52871954
Epoch:  210 MSE= 0.5267403
Epoch:  240 MSE= 0.5256731
Epoch:  270 MSE= 0.52508485
Epoch:  300 MSE= 0.52475566
[[ 2.0685577 ]
 [ 0.8509807 ]
 [ 0.12402568]
 [-0.30351254]
 [ 0.33606333]
 [-0.00286167]
 [-0.04022788]
 [-0.8413803 ]
 [-0.81440705]]


# Feeding Data to the Training Algorithm

In [15]:
tf.reset_default_graph()
A=tf.placeholder(tf.float32,shape=(None,3))
B=A+5
with tf.Session() as sess:
    print(B.eval(feed_dict={A:[[1,2,3]]}))

[[6. 7. 8.]]


In [16]:
tf.reset_default_graph()
X=tf.placeholder(tf.float32,shape=(None,n+1))
y=tf.placeholder(tf.float32,shape=(None,1))
batch_size=100
n_epochs=10
learning_rate=0.01
n_batches=int(np.ceil(m/batch_size))

theta=tf.Variable(tf.random_uniform([n+1,1],-1,1))
y_pred=tf.matmul(X,theta)
error=y_pred-y
mse=tf.reduce_mean(tf.square(error))
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(mse)

In [17]:
init =tf.global_variables_initializer()

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

In [18]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch,y_batch=fetch_batch(epoch,batch_index,batch_size)
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
    print(theta.eval())    

[[ 2.070146  ]
 [ 0.8378247 ]
 [ 0.11930648]
 [-0.26163596]
 [ 0.34127533]
 [ 0.00381156]
 [-0.01166134]
 [-0.8841386 ]
 [-0.849708  ]]


# Saving and Restoring Models

In [19]:
tf.reset_default_graph()
n_epochs=100
learning_rate=0.1
X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32)
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32)
theta=tf.Variable(tf.random_uniform([n+1,1],-1,1))
y_pred=tf.matmul(X,theta)
error=y-y_pred
mse=tf.reduce_mean(tf.square(error))
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(mse)

init=tf.global_variables_initializer()
saver=tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        sess.run(training_op)
        if epoch %10==0:
            saver.save(sess,'C:/Users/xiekch/Documents/ML/handson-ml/my/tmp/my_model.ckpt')
    best_theta=theta.eval()
    saver.save(sess,'C:/Users/xiekch/Documents/ML/handson-ml/my/tmp/my_model_final.ckpt')
print(best_theta)

[[ 2.0685577e+00]
 [ 6.9473344e-01]
 [ 1.2223595e-01]
 [ 4.3138064e-02]
 [ 2.6788551e-02]
 [-1.4847987e-03]
 [-3.6744937e-02]
 [-1.0011474e+00]
 [-9.5349991e-01]]


In [20]:
with tf.Session() as sess:
    saver.restore(sess,'C:/Users/xiekch/Documents/ML/handson-ml/my/tmp/my_model_final.ckpt')
    best_theta_restored=theta.eval()
print(best_theta_restored)

INFO:tensorflow:Restoring parameters from C:/Users/xiekch/Documents/ML/handson-ml/my/tmp/my_model_final.ckpt
[[ 2.0685577e+00]
 [ 6.9473344e-01]
 [ 1.2223595e-01]
 [ 4.3138064e-02]
 [ 2.6788551e-02]
 [-1.4847987e-03]
 [-3.6744937e-02]
 [-1.0011474e+00]
 [-9.5349991e-01]]


In [22]:
#best_theta==best_theta_restored

# TensorBoard

In [31]:
from datetime import datetime
now=datetime.now().strftime('%Y%m%d%H%M%S')
root_logdir='tf_logs'
logdir='{}/run-{}/'.format(root_logdir,now)

In [32]:
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [33]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [35]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [36]:
with tf.Session() as sess:                                                        # not shown in the book
    sess.run(init)                                                                # not shown

    for epoch in range(n_epochs):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()     
file_writer.close()

In [41]:
#PS C:\Users\xiekch\Documents\ML\handson-ml\my\tf_logs> tensorboard --logdir="run-2018081
#2095636"

# Modularity

# Sharing Variables

In [52]:
with tf.variable_scope('relu'):
    threshold=tf.get_variable('threshold',shape=(),initializer=tf.constant_initializer(0))

In [53]:
with tf.variable_scope('relu',reuse=True):
    threshold=tf.get_variable('threshold')
    print(threshold)

<tf.Variable 'relu/threshold:0' shape=() dtype=float32_ref>


In [55]:
with tf.variable_scope('relu') as scope:
    scope.reuse_variables()
    threshold=tf.get_variable('threshold')
    print(threshold)

<tf.Variable 'relu/threshold:0' shape=() dtype=float32_ref>
